# Projet d'optimisation 
## Par Pierre Sion et Rania Fathi

## I- Modélisation 

### Question 1 :
On définit l'élasticité-prix comme étant la variation relative de la quantité demandée par rapport au prix, soit encore: 
    $\varepsilon_i = \frac{\frac{dx_i}{x_i}}{\frac{dp_i}{p_i}} $ 
où l'on a noté $x_i$ la demande et $p_i$ le prix du ième produit 
    
Cela peut encore s'écrire: 
$\varepsilon_i = \frac{\partial x_i}{\partial p_i} \frac{p_i}{x_i}$

Comme on s'attend à ce que la demande pour un produit diminue si le prix augmente, on a $\varepsilon_i \leq 0$

Pour ce qui est de l'élasticité prix croisée entre le gouda et l'edam, elle vaut: 

$\varepsilon_c = \frac{\frac{dx_g}{x_g}}{\frac{dp_e}{p_e}} = \frac{\partial x_g}{\partial p_e} \frac{p_e}{x_g}$

Les biens considérés étant substituables, on a $\varepsilon_c \geq 0$ et cela nous donne la relation 

### Question 2 :
Le lait brut possède une quantité de matière grasse totale répartie entre les quatre produits, que l'on note $M$. En notant $m_i$ la teneur en matière grasse présente dans chaque produit, on a : 

$ \sum_{i=1}^{4}m_i x_i \leq M X$

On fait par ailleurs le choix de mettre une inégalité plutôt qu'une égalité pour considérer le cas où tout le lait n'est pas transformé 

### Question 3 : 
De la même manière, on note $L$ la quantité totale de lactose contenue dans le lait, et $l_i$ celle contenue dans chaque produit, et on a alors la contrainte suivante: 

$\sum_{i=1}^{4} l_i \leq L$

### Question 5 :
On dispose donc du problème d'optimisation sous contraintes suivant : $\max_{c(x) \leq 0} \Pi (x)$ 

où le profit $\Pi:\mathbf{R}^4 \times \mathbf{R}^4 \rightarrow \mathbf{R}^+  $ vaut $\Pi(x,p) = \sum_{i=1}^4 p_i x_i $ (En toute rigueur, $\Pi$ est égal au revenu plutôt qu'au profit, mais l'omission du terme $-C$ correspondant au coût total du lait acheté ne modifie pas le problème d'optimisation à résoudre)

et les contraintes $c : \mathbf{R}^4 \times \mathbf{R}^4 \rightarrow \mathbf{R}^7$ sont donc les suivantes : 

$c_1(x,p) = x_1 \\ 
c_2(x,p) = x_2\\ 
c_3(x,p) = x_3\\ 
c_4(x,p) = x_4\\ 
c_5(x,p) = \\ 
c_6(x,p) = \\ 
c_7(x,p) =  $

## II- Implémentation 